In [8]:
import pymongo
import pandas as pd
import re

client = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = client["Vulns"]

teams_col = mydb["teams"]
assets_col = mydb["assets"]
export_col = mydb["export"]

Calcular todas as vulnerabilidades em máquinas de ACR > 9 e entender qual o time que possui maior quantidade de vulnerabilidades, separando por tipos de vulnerabilidade e total juntando todas.

In [6]:
result = export_col.aggregate([
    {
        '$match': {
            'asset.ACR': {
                '$gte': 9
            }
        }
    }, {
        '$unwind': '$asset.team'
    }, {
        '$group': {
            '_id': '$asset.team', 
            'Critical': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'Critical'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'High': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'High'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'Medium': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'Medium'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'Low': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'Low'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'Info': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'Info'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'Total': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'Critical': -1, 
            'High': -1, 
            'Medium': -1, 
            'Low': -1
        }
    }
])

list(result)

[{'_id': 'Morocco',
  'Critical': 5557,
  'High': 8278,
  'Medium': 13930,
  'Low': 13695,
  'Info': 289,
  'Total': 41749},
 {'_id': 'Bhutan',
  'Critical': 3115,
  'High': 4628,
  'Medium': 7747,
  'Low': 7628,
  'Info': 163,
  'Total': 23281},
 {'_id': 'Malaysia',
  'Critical': 3096,
  'High': 4466,
  'Medium': 7488,
  'Low': 7219,
  'Info': 155,
  'Total': 22424},
 {'_id': 'Kazakhstan',
  'Critical': 3082,
  'High': 4630,
  'Medium': 7738,
  'Low': 7630,
  'Info': 165,
  'Total': 23245},
 {'_id': 'Malta',
  'Critical': 2952,
  'High': 4163,
  'Medium': 7052,
  'Low': 7126,
  'Info': 159,
  'Total': 21452},
 {'_id': 'Mauritius',
  'Critical': 2946,
  'High': 4235,
  'Medium': 7235,
  'Low': 7158,
  'Info': 154,
  'Total': 21728},
 {'_id': 'Australia',
  'Critical': 2926,
  'High': 4383,
  'Medium': 7333,
  'Low': 7211,
  'Info': 149,
  'Total': 22002},
 {'_id': 'France',
  'Critical': 2901,
  'High': 4387,
  'Medium': 7226,
  'Low': 7227,
  'Info': 173,
  'Total': 21914},
 {'_id': '

In [14]:
times = client['Vulns']['teams'].aggregate([
    {
        '$match': {
            'members': {
                '$elemMatch': {
                    '$regex': re.compile(r"(?i)[aeiou]$")
                }
            }
        }
    }, {
        '$project': {
            '_id': 0, 
            'name': 1
        }
    }
])

times = [x["name"] for x in list(times)]

result2 = client['Vulns']['export'].aggregate([
    {
        '$match': {
            '$and': [
                {
                    'vuln.pub_date': re.compile(r"^2012")
                }, {
                    'asset.ip': re.compile(r"^10")
                }, {
                    'asset.team': {
                        '$in': times
                    }
                }
            ]
        }
    }, {
        '$group': {
            '_id': '$asset.ip', 
            'total': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'total': -1
        }
    }
])

list(result2)

[{'_id': '10.150.152.142', 'total': 12},
 {'_id': '10.88.190.69', 'total': 12},
 {'_id': '10.13.91.60', 'total': 11},
 {'_id': '10.197.84.173', 'total': 10},
 {'_id': '10.159.213.67', 'total': 9},
 {'_id': '10.49.212.100', 'total': 9},
 {'_id': '10.56.212.0', 'total': 9},
 {'_id': '10.140.54.30', 'total': 9},
 {'_id': '10.213.192.206', 'total': 8},
 {'_id': '10.11.140.129', 'total': 8},
 {'_id': '10.200.41.123', 'total': 8},
 {'_id': '10.75.55.115', 'total': 8},
 {'_id': '10.116.51.69', 'total': 8},
 {'_id': '10.222.150.115', 'total': 8},
 {'_id': '10.233.135.147', 'total': 8},
 {'_id': '10.127.89.153', 'total': 8},
 {'_id': '10.35.189.13', 'total': 8},
 {'_id': '10.159.74.183', 'total': 7},
 {'_id': '10.109.156.40', 'total': 7},
 {'_id': '10.134.153.87', 'total': 7},
 {'_id': '10.152.214.148', 'total': 7},
 {'_id': '10.87.82.68', 'total': 7},
 {'_id': '10.205.15.193', 'total': 7},
 {'_id': '10.166.39.52', 'total': 7},
 {'_id': '10.84.253.119', 'total': 7},
 {'_id': '10.132.88.222', 't